In [115]:
%%time
import npm_crawler as npm
reload(npm)

packages = dict(npm.get_most_starred_packages(max_pages=10))
print packages.values()


[u'jashkenas/coffeescript', u'jonkemp/gulp-useref', u'chjj/blessed', u'jaredhanson/passport-local', u'wycats/handlebars.js', u'less/less.js', u'sindresorhus/gulp-rev', u'plus3network/gulp-less', u'cheeriojs/cheerio', u'voodootikigod/node-serialport', u'stylus/stylus', u'mashpie/i18n-node', u'Marak/colors.js', u'jshint/jshint', u'thomseddon/node-oauth2-server', u'Automattic/mongoose', u'hparra/gulp-rename', u'sindresorhus/gulp-filter', u'aheckmann/gm', u'gruntjs/grunt-contrib-watch', u'mbostock/d3', u'Automattic/cli-table', u'DaftMonk/generator-angular-fullstack', u'shtylman/node-uuid', u'addyosmani/psi', u'Medium/phantomjs', u'fluent-ffmpeg/node-fluent-ffmpeg', u'mscdex/busboy', u'shouldjs/should.js', u'gulpjs/gulp', u'jonschlinkert/gulp-htmlmin', u'substack/tape', u'sun-zheng-an/gulp-shell', u'motdotla/dotenv', u'ben-eb/gulp-uncss', u'facebook/immutable-js', u'etsy/statsd', u'sindresorhus/gulp-ruby-sass', u'Microsoft/TypeScript', u'Leonidas-from-XIV/node-xml2js', u'jprichardson/string